# Modern Python Async Addendum

This addendum connects the generator-based coroutine scheduler from the original concurrency lectures with modern Python’s `async def`, `await`, and the `asyncio` event loop.

The goal is to show that modern async is not a new model, but a standardized notation and library over the same coroutine/scheduler ideas you already built.

## 1. Coroutines Revisited: From Generators to `async def`

Earlier, a coroutine was implemented as a generator:

```python
def child_old():
    print("start")
    yield
    print("finish")
```

Modern Python uses `async def` to define a coroutine:

```python
async def child_new():
    print("start")
    await something
    print("finish")
```

Both represent a computation that can suspend and later resume. The main difference is notation and who drives the coroutine (your scheduler vs. the event loop).

## 2. Suspending Execution: `yield` and a Modern Equivalent

A generator-based coroutine suspends using a bare `yield`:

```python
yield
```

To express the same kind of suspension in modern async code, we define a tiny awaitable whose `__await__()` method yields exactly once:

```python
class YieldAwaitable:
    def __await__(self):
        yield              # single suspension point
        return None
```

Now the modern equivalent of a bare suspension is:

```python
await YieldAwaitable()
```

You can think of `await YieldAwaitable()` as “just yield control back to the event loop, and resume me later,” exactly like `yield` did for your custom scheduler.

## 3. From System Calls to Awaitables

In the earlier scheduler, coroutines yielded `SystemCall` objects, which the scheduler interpreted:

```python
tid = yield GetTid()
```

The scheduler saw the `GetTid` object, filled in `task` and `sched` fields on it, and then resumed the task with a value via `send()`.

In modern Python, this corresponds to awaiting an object that implements the *Awaitable protocol*: its `__await__()` method returns a generator. For example, a modern version of `GetTid` might look like this:

```python
import asyncio

class GetTidAwaitable:
    def __await__(self):
        task = asyncio.current_task()
        tid  = task.get_name() if task is not None else None
        yield                # suspend once
        return tid
```

Using it:

```python
tid = await GetTidAwaitable()
```

This mirrors the behavior of the old `GetTid` system call: it queries “who am I?” from the runtime and resumes with that value.

## 4. Spawning and Joining Tasks with Awaitables

The earlier scheduler supported:

- `yield NewTask(coro)` to spawn a child task
- `yield WaitTask(tid)` to join on a child task

We can express the same structure in modern async using two tiny awaitables.

### `NewTaskAwaitable`: spawn a child

```python
class NewTaskAwaitable:
    def __init__(self, coro):
        self.coro = coro

    def __await__(self):
        task = asyncio.create_task(self.coro())
        yield                  # cooperative suspension
        return task
```

This corresponds to:

```python
child = yield NewTask(foo())
```

Becoming:

```python
child = await NewTaskAwaitable(foo_new)
```

where `foo_new` is an `async def` coroutine.

### `WaitTaskAwaitable`: join a child

```python
class WaitTaskAwaitable:
    def __init__(self, task):
        self.task = task

    def __await__(self):
        # run the child's await logic to completion
        result = yield from self.task.__await__()
        return result
```

This corresponds directly to the old join:

```python
yield WaitTask(child_tid)
```

becoming:

```python
await WaitTaskAwaitable(child)
```

Both mean: “suspend me until this child completes, then resume with its result.”

## 5. A Short Note on `yield from`: Calling Another Generator

A generator is a function that can pause. If one generator wants to run another generator and obtain its return value, writing:

```python
return other_generator()
```

only *creates* the generator—it does not execute it.

To “call” another generator and run it to completion, Python uses:

```python
yield from other_generator
```

This behaves like a function call:

- execution enters `other_generator`,
- it pauses whenever it yields,
- it finishes normally,
- its return value becomes the caller’s return value.

In other words:

```python
yield from g
```

is to generators what

```python
return g()
```

is to normal functions.

This is why `WaitTaskAwaitable` uses `yield from self.task.__await__()`: it “calls” the child task’s await generator and returns its result, just like joining a child in the old scheduler.

## 6. Example: Spawning and Joining with Awaitables

Putting it all together:

```python
import asyncio

class YieldAwaitable:
    def __await__(self):
        yield
        return None

class NewTaskAwaitable:
    def __init__(self, coro):
        self.coro = coro

    def __await__(self):
        task = asyncio.create_task(self.coro())
        yield
        return task

class WaitTaskAwaitable:
    def __init__(self, task):
        self.task = task

    def __await__(self):
        result = yield from self.task.__await__()
        return result

class GetTidAwaitable:
    def __await__(self):
        task = asyncio.current_task()
        tid  = task.get_name() if task is not None else None
        yield
        return tid

async def foo_new():
    for i in range(3):
        tid = await GetTidAwaitable()
        print("I'm foo in task", tid)
        await YieldAwaitable()   # like a bare 'yield' in the old model

async def main_new():
    child = await NewTaskAwaitable(foo_new)
    print("Waiting for child")
    await WaitTaskAwaitable(child)
    print("Child done")

asyncio.run(main_new())
```

This example mirrors the earlier SystemCall-based spawn + join, but uses modern async syntax and the event loop instead of the custom scheduler.

## 7. The Event Loop in Place of the Manual Scheduler

In the earlier model, you started the scheduler manually:

```python
sched = Scheduler()
sched.new(main_old())
sched.mainloop()
```

In modern Python, you write:

```python
asyncio.run(main_new())
```

In both cases:

- a main coroutine starts the computation,
- the runtime schedules tasks cooperatively,
- suspension points (`yield` or `await`) transfer control back to the scheduler/event loop,
- and tasks are resumed later.

The names changed, but the structure is the same: a loop drives coroutines, and coroutines suspend themselves explicitly.

## 8. Putting It All Together

Here is a concise mapping between the earlier generator-based model and modern async:

| Idea              | Earlier model                        | Modern Python                         |
|-------------------|--------------------------------------|---------------------------------------|
| Coroutine         | generator function                   | `async def`                           |
| Suspend           | `yield`                              | `await`                               |
| Bare yield        | `yield`                              | `await YieldAwaitable()`              |
| System call       | `yield SystemCall()`                 | `await Awaitable()`                   |
| Spawn task        | `yield NewTask(coro)`                | `await NewTaskAwaitable(coro)`        |
| Join task         | `yield WaitTask(tid)`                | `await WaitTaskAwaitable(task)`       |
| Get task id       | `yield GetTid()`                     | `await GetTidAwaitable()`             |
| Scheduler         | custom `Scheduler.mainloop()`        | asyncio event loop                    |
| Entry point       | `sched.new(main); sched.mainloop()`  | `asyncio.run(main)`                   |

Modern Python introduces dedicated syntax and a built-in event loop, but the underlying coroutine model remains the same as the generator-based scheduler you built earlier.

## 9. Echo Server in Modern Async Python

Earlier, you implemented an echo server using your own coroutine scheduler: nonblocking sockets, SystemCalls like `ReadWait` and `WriteWait`, and a main loop that used `select()` to decide which tasks to resume.

We now show the same idea expressed using modern async Python. The goal is to see how the concepts map directly to `async def`, `await`, and the `asyncio` event loop.

### 9.1 What Is a Server?

A server is a program that:

1. Creates a listening socket on some address (IP + port).
2. Waits for clients to connect.
3. For each client connection, creates a per-client handler.
4. Each handler receives and sends bytes on its connection.
5. The server stays alive to accept more connections.

The reason this requires concurrency is that multiple clients may connect at the same time. If the server blocked while handling one client, all others would wait. The earlier coroutine scheduler solved this by allowing each client handler to suspend while waiting for I/O and letting the scheduler run other handlers in the meantime.

### 9.2 Stream Readers and Writers in `asyncio`

In modern async Python, `asyncio.start_server` automates the low-level socket work. For every new client connection, it constructs two helper objects:

- a `StreamReader` for receiving data,
- a `StreamWriter` for sending data.

It then calls your handler coroutine with these two objects:

```python
async def handle_client(reader, writer):
    ...
```

The handler uses:

- `await reader.read(n)` to wait for incoming data,
- `writer.write(data)` to enqueue outgoing bytes,
- `await writer.drain()` to wait until the data is sent.

These correspond conceptually to the earlier SystemCalls:

- `yield ReadWait(sock)`
- `yield WriteWait(sock, data)`

but without manually working with `select()` or nonblocking socket flags.

### 9.3 Modern Echo Handler

A modern echo handler looks like this:

```python
import asyncio

async def handle_client(reader, writer):
    addr = writer.get_extra_info("peername")
    print("connection from", addr)

    while True:
        data = await reader.read(1024)   # like ReadWait
        if not data:
            break

        writer.write(data)               # enqueue bytes to send back
        await writer.drain()             # like WriteWait

    writer.close()
    await writer.wait_closed()
    print("connection closed", addr)
```

This coroutine handles one client connection:

- it loops, reading data from the client,
- echoes the same data back,
- and exits when the client disconnects.

### 9.4 Modern Server Loop

The earlier scheduler-driven server loop looked roughly like:

```python
def server_old(port):
    # create listening socket
    # accept connections
    # wrap each new client in a coroutine
    # schedule each coroutine with NewTask
    ...
```

The modern version is:

```python
import asyncio

async def main():
    server = await asyncio.start_server(
        handle_client,
        host="127.0.0.1",
        port=25000,
    )
    print("server running on 127.0.0.1:25000")
    await server.serve_forever()     # event loop accepts clients and dispatches handlers

asyncio.run(main())
```

Here:

- `asyncio.start_server` creates the listening socket and accepts new connections.
- For each new client, it starts a new task running `handle_client(reader, writer)`.
- `server.serve_forever()` keeps accepting new clients until the program is stopped.
- `asyncio.run(main())` starts the event loop and runs the main coroutine, just as your old `Scheduler.mainloop()` drove the generator-based coroutines.

### 9.5 Correspondence with the Earlier Echo Server

Here is how the echo server concepts line up:

| Idea               | Earlier coroutine model             | Modern async Python                               |
|--------------------|-------------------------------------|---------------------------------------------------|
| Per-client handler | generator coroutine                 | `async def handle_client(reader, writer)`         |
| Read when ready    | `yield ReadWait(sock)`              | `await reader.read(...)`                          |
| Write when ready   | `yield WriteWait(sock, data)`       | `writer.write(data); await writer.drain()`        |
| New connection     | manual `accept()` + `NewTask`       | automatic via `asyncio.start_server`              |
| Task scheduling    | custom `Scheduler.mainloop()`       | `asyncio` event loop                              |
| Main entry         | `sched.new(main); sched.mainloop()` | `asyncio.run(main)`                               |

The structure is the same: coroutines handle clients and suspend while waiting for I/O. Modern async Python simply provides a built-in event loop and stream helpers instead of SystemCalls and a custom scheduler.